### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [268]:
# Dependencies and Setup
import pandas as pd
import os
import numpy as np

# File to Load (Remember to Change These)
load_file_path = os.path.join("..", "Resources", "purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
main_df = pd.read_csv(load_file_path)

# Test loading of CSV, view file layout
main_df.head()

Purchase ID             SN  Age Gender  Item ID  \
0            0        Lisim78   20   Male      108   
1            1    Lisovynya38   40   Male      143   
2            2     Ithergue48   24   Male       92   
3            3  Chamassasya86   24   Male      100   
4            4      Iskosia90   23   Male      131   

                                   Item Name  Price  
0  Extraction, Quickblade Of Trembling Hands   3.53  
1                          Frenzied Scimitar   1.56  
2                               Final Critic   4.88  
3                                Blindscythe   3.27  
4                                       Fury   1.44

In [269]:
# Verify that no missing values exist in df
main_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

## Player Count

* Display the total number of players


In [270]:
# OPTIONAL: Create narrow df with only 2 columns to prep for simplified groupby
count_df = main_df.loc[:, ['Purchase ID', 'SN']]
count_df.head()

Purchase ID             SN
0            0        Lisim78
1            1    Lisovynya38
2            2     Ithergue48
3            3  Chamassasya86
4            4      Iskosia90

In [271]:
# Group narrowed df by Screen Name and generate a count
player_df = count_df.groupby(["SN"]).count()
player_df

Purchase ID
SN                        
Adairialis76             1
Adastirin33              1
Aeda94                   1
Aela59                   1
Aelaria33                1
...                    ...
Yathecal82               3
Yathedeu43               2
Yoishirrala98            1
Zhisrisu83               2
Zontibe81                3

[576 rows x 1 columns]

In [272]:
# Count rows in grouped table to generate count of Screen Names (i.e., unique players)
total_players = len(player_df)
total_players

# Create summary table showing player count
summary_players = pd.DataFrame({"Total Players": total_players}, index=[0])
summary_players

Total Players
0            576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [273]:
# Use main_df to calculate total purchases
total_purchases = len(main_df)
total_purchases

# Use main_df to calculate number of unique items, and total revenue
item_df = main_df.groupby(['Item ID'])
total_items = len(item_df)
total_items

179

In [274]:
# Use main_df to calculate average price
average_price = main_df["Price"].mean()
average_price

# Use main_df to calculate total revenue
total_rev = main_df["Price"].sum()
total_rev

# Create summary table showing purchasing analysis values
summary_purchasing = pd.DataFrame({"Total Purchases": total_purchases,
                                  "Number of Unique Items": total_items,
                                  "Average Price": average_price,
                                  "Total Revenue": total_rev},
                                 index=[0])
summary_purchasing

Total Purchases  Number of Unique Items  Average Price  Total Revenue
0              780                     179       3.050987        2379.77

## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [275]:
# Use main_df to calculate gender stats

# Narrow main_df to relevant columns
# loc_gender_df = main_df.loc[:, ["SN", "Gender"]]
# loc_gender_df

# Remove duplicate Screen Names from df, reindex in order to avoid Setting with Copy warning for future operations on df
unique_gender_df = main_df.drop_duplicates(subset="SN", keep='first').reset_index(drop=True)
# unique_gender_df

# Group by Gender to get number of male/female
gender_df = unique_gender_df.groupby(['Gender'])
gender_count = gender_df.count()
gender_count

# Generate percentages list (?)
percent_list = (gender_count["SN"]/total_players) * 100
percent_list

# Create summary table showing gender demographics values
summary_gender = pd.DataFrame({"Total Players": gender_count["Purchase ID"],
                              "Percentage of Players": percent_list})
summary_gender

Total Players  Percentage of Players
Gender                                                     
Female                            81              14.062500
Male                             484              84.027778
Other / Non-Disclosed             11               1.909722


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [276]:
# Create a df grouped by gender
gender_pur_df = main_df.groupby("Gender")

# Calculate the total purchases by gender
pur_count_gender = gender_pur_df["Purchase ID"].count()
pur_count_gender

# Calculate the average purchase price by gender
avg_price_gender = gender_pur_df["Price"].mean()
avg_price_gender

# Calculate the total purchase value by gender
total_pur_gender = gender_pur_df["Price"].sum()
total_pur_gender

# Calculate the average total purchases per person by gender
gender_group_df = main_df.groupby(["SN"])
# gender_group_df

gpur_per_pers_df = gender_group_df["Price"].sum()
# gpur_per_pers_df

merge_gpurch_df = pd.merge(unique_gender_df, gpur_per_pers_df, how='left', on='SN')
# merge_gpurch_df

gmerge_group_df = merge_gpurch_df.groupby(['Gender'])
gpurch_per_pers_gen = gmerge_group_df["Price_y"].mean()
# gpurch_per_pers_gen

# Create a summary table showing purchase stats by gender
summary_purch_gender = pd.DataFrame({"Total Purchases": pur_count_gender,
                               "Average Purchase Price": avg_price_gender,
                               "Total Purchase Value": total_pur_gender,
                               "Avg Total Purchase per Person": gpurch_per_pers_gen})
summary_purch_gender

Total Purchases  Average Purchase Price  \
Gender                                                           
Female                             113                3.203009   
Male                               652                3.017853   
Other / Non-Disclosed               15                3.346000   

                       Total Purchase Value  Avg Total Purchase per Person  
Gender                                                                      
Female                               361.94                       4.468395  
Male                                1967.64                       4.065372  
Other / Non-Disclosed                 50.19                       4.562727

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [277]:
# Create bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 120]

# Establish bin names
bin_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Add age range column to df by cutting the df by age
unique_gender_df["Age Range"] = pd.cut(unique_gender_df["Age"], bins, labels=bin_names)
unique_gender_df.head()

# Create a new df grouped by the age range column and calculate the total players in each age range
age_df = unique_gender_df.groupby(['Age Range'])
count_age = age_df["Gender"].count()
# count_age

# Calculate the percentage of players in each age range
percent_age = (count_age / total_players) * 100

# Create a summary table showing age breakdown of players
summary_age = pd.DataFrame({"Total Players": count_age,
                           "Percentage of Players": percent_age})
summary_age

Total Players  Percentage of Players
Age Range                                      
<10                   17               2.951389
10-14                 22               3.819444
15-19                107              18.576389
20-24                258              44.791667
25-29                 77              13.368056
30-34                 52               9.027778
35-39                 31               5.381944
40+                   12               2.083333

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [278]:
# Create an age range column in the full (non-unique) df by cutting the df by age
main_df["Age Range"] = pd.cut(main_df["Age"], bins, labels=bin_names)
# main_df()

# Create a new df grouped by age range
age_group_df = main_df.groupby(['Age Range'])

# Calculate the total purchases by age range
purchase_age = age_group_df["Purchase ID"].count()
# purchase_age

# Calculate the average purchase price by age range
avgprice_age = age_group_df["Price"].mean()
# avgprice_age

# Calculate the total purchases by age range
totalpurch_age = age_group_df["Price"].sum()
# totalpurch_age

# Calculate the average total purchases per person by age range
age_group_df = main_df.groupby(["SN"])
# age_group_df

pur_per_pers_df = age_group_df["Price"].sum()
# pur_per_pers_df

merge_purch_df = pd.merge(unique_gender_df, pur_per_pers_df, how='left', on='SN')
# merge_purch_df

merge_group_df = merge_purch_df.groupby(['Age Range'])
purch_per_pers_gen = merge_group_df["Price_y"].mean()
# purch_per_pers_gen

# Create a summary table showing purchase stats by age
summary_purch_age = pd.DataFrame({"Purchase Count": purchase_age,
                           "Average Purchase Price": avgprice_age,
                                 "Total Purchase Value": totalpurch_age,
                                 "Avg Total Purchase per Person": purch_per_pers_gen})
summary_purch_age

Purchase Count  Average Purchase Price  Total Purchase Value  \
Age Range                                                                 
<10                    23                3.353478                 77.13   
10-14                  28                2.956429                 82.78   
15-19                 136                3.035956                412.89   
20-24                 365                3.052219               1114.06   
25-29                 101                2.900990                293.00   
30-34                  73                2.931507                214.00   
35-39                  41                3.601707                147.67   
40+                    13                2.941538                 38.24   

           Avg Total Purchase per Person  
Age Range                                 
<10                             4.537059  
10-14                           3.762727  
15-19                           3.858785  
20-24                           4.318062  
25-29                           3.805195  
30-34                           4.115385  
35-39                           4.763548  
40+                             3.186667

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [279]:
# Calculate number of purchases by individual
count_purch_top = gender_group_df.count()["Price"]
count_purch_top

# Calculate average purchase price by individual
avg_price_top = gender_group_df.mean()["Price"]
avg_price_top

# Calculate total purchase value by individual
total_purchval_top = gender_group_df.sum()["Price"]
total_purchval_top

# Create new df with requested top spenders stats
top_spend = pd.DataFrame({"Purchase Count": count_purch_top,
                           "Average Purchase Price": avg_price_top,
                                 "Total Purchase Value": total_purchval_top})
top_spend

# Sort new df to show top spenders
sort_top_spend = top_spend.sort_values("Total Purchase Value", ascending=False)
sort_top_spend.head()

Purchase Count  Average Purchase Price  Total Purchase Value
SN                                                                       
Lisosia93                 5                3.792000                 18.96
Idastidru52               4                3.862500                 15.45
Chamjask73                3                4.610000                 13.83
Iral74                    4                3.405000                 13.62
Iskadarya95               3                4.366667                 13.10

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [280]:
# Create narrow dataframe with 3 relevant columns
items_df = main_df.loc[:, ["Item ID", "Item Name", "Price"]]
items_df

# Calculate the total purchase value by item
total_purchval_by_item = items_df.groupby(["Item ID", "Item Name"]).sum()["Price"]
total_purchval_by_item

# Calculate the purchase count by item
purch_count_by_item = items_df.groupby(["Item ID", "Item Name"]).count()["Price"]
purch_count_by_item

# Calculate the average price by item
avg_price_by_item = items_df.groupby(["Item ID", "Item Name"]).mean()["Price"]
avg_price_by_item

# Create new df with requested stats
pop_items = pd.DataFrame({"Purchase Count": purch_count_by_item,
                           "Item Price": avg_price_by_item,
                                 "Total Purchase Value": total_purchval_by_item})
pop_items

# Sort new df to show most popular items
sort_pop_items = pop_items.sort_values("Purchase Count", ascending=False)
sort_pop_items.head()

Purchase Count  \
Item ID Item Name                                                      
92      Final Critic                                              13   
178     Oathbreaker, Last Hope of the Breaking Storm              12   
145     Fiery Glass Crusader                                       9   
132     Persuasion                                                 9   
108     Extraction, Quickblade Of Trembling Hands                  9   

                                                      Item Price  \
Item ID Item Name                                                  
92      Final Critic                                    4.614615   
178     Oathbreaker, Last Hope of the Breaking Storm    4.230000   
145     Fiery Glass Crusader                            4.580000   
132     Persuasion                                      3.221111   
108     Extraction, Quickblade Of Trembling Hands       3.530000   

                                                      Total Purchase Value  
Item ID Item Name                                                           
92      Final Critic                                                 59.99  
178     Oathbreaker, Last Hope of the Breaking Storm                 50.76  
145     Fiery Glass Crusader                                         41.22  
132     Persuasion                                                   28.99  
108     Extraction, Quickblade Of Trembling Hands                    31.77

## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [281]:
# Sort most popular items table by total purchase value to show most profitable items
sort_prof_items = pop_items.sort_values("Total Purchase Value", ascending=False)
sort_prof_items.head()

Purchase Count  \
Item ID Item Name                                                      
92      Final Critic                                              13   
178     Oathbreaker, Last Hope of the Breaking Storm              12   
82      Nirvana                                                    9   
145     Fiery Glass Crusader                                       9   
103     Singed Scalpel                                             8   

                                                      Item Price  \
Item ID Item Name                                                  
92      Final Critic                                    4.614615   
178     Oathbreaker, Last Hope of the Breaking Storm    4.230000   
82      Nirvana                                         4.900000   
145     Fiery Glass Crusader                            4.580000   
103     Singed Scalpel                                  4.350000   

                                                      Total Purchase Value  
Item ID Item Name                                                           
92      Final Critic                                                 59.99  
178     Oathbreaker, Last Hope of the Breaking Storm                 50.76  
82      Nirvana                                                      44.10  
145     Fiery Glass Crusader                                         41.22  
103     Singed Scalpel                                               34.80